
# POC 激活验证

### 1. 同构激活图

In [1]:
import torch 
params = torch.load('./shadow_model_ckpt/mnist/models2/shadow_benign_0.model')
from model_lib.mnist_cnn_model import Model2 as Model
m = Model()
m.load_state_dict(params)
print(m)

/home/dorian/.conda/envs/mntd/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model2(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (fc): Linear(in_features=576, out_features=256, bias=True)
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (output): Linear(in_features=120, out_features=10, bias=True)
)


In [2]:
import dgl
import torch
from model_lib.hetero_struct_backdoor_dataset import HeteroStrucBackdoorDataset
from utils_gnn import all_u_to_v


def cnn2graph_activation(model, model_info):
    # convert cnn model to a dgl graph
    # model: model weight data
    # model_info: model struct info
    
    layers = []
    all_edges = []


    node_layer_idx = []
    all_node_feats = []
    pre_node_size = []
    node_params = []
    all_bias_feats = []
    pre_bias_size = []
    
    cnt = 0
    with torch.no_grad():
        for i in range(len(model_info)):
            cur_layer_info = model_info[i]
            cur_layer_node = []
            
            if 'conv' in cur_layer_info['name']:
                # construct cur layer nodes
                for weight, bias in zip(model.get_submodule(cur_layer_info['name']).weight, 
                                        model.get_submodule(cur_layer_info['name']).bias):
                    # print("cur layer info:", model.get_submodule(cur_layer_info['name']))
                    cur_layer_node.append(cnt)
                    cnt += 1
                    node_layer_idx.append(i)
                    # params 
                    conv = model.get_submodule(cur_layer_info['name'])
                    params = [conv.kernel_size, conv.stride, conv.padding]
                    node_params.append(params)
                    # print(params)
                    # featue resize?
                    w = weight[0]
                    r, c = w.size()
                    pre_node_size.append([r, c])
                    all_node_feats.append(padding(w, 512, 512))
                    b = bias.expand(1,1)
                    r, c = b.size()
                    # print("size:", b.size())
                    pre_bias_size.append([r, c])
                    all_bias_feats.append(padding(b, 1, 512))
                    # print("conv weight:", w.shape)
                    # bias
                    # print("conv bias:", bias)
            else:
                # construct dense layer node
                cur_layer_node.append(cnt)
                cnt += 1
                node_layer_idx.append(i)
                params = [(0, 0), (0, 0), (0, 0)]
                node_params.append(params)
                # feature resize?
                weight = model.get_submodule(cur_layer_info['name']).weight.t()
                bias = model.get_submodule(cur_layer_info['name']).bias
                w = weight
                r, c = w.size()
                pre_node_size.append([r, c])
                all_node_feats.append(padding(w, 512, 512))
                b = bias.expand(1,-1)
                r, c = b.size()
                # print("size:", b.size())
                pre_bias_size.append([r, c])
                all_bias_feats.append(padding(b, 1, 512))
            layers.append(cur_layer_node)
            
    # get all edges
    for idx in range(len(layers)):
        if idx < len(layers) - 1:
            edges = all_u_to_v(layers[idx], layers[idx+1])
            all_edges += edges
    
    all_edges = torch.tensor(all_edges).t()
    u, v = all_edges[0], all_edges[1]
    g = dgl.graph((u,v)).to('cuda')
    g.ndata['x'] = torch.stack(all_node_feats)
    g.ndata['idx'] = torch.tensor(node_layer_idx).to('cuda')
    g.ndata['node_size'] = torch.tensor(pre_node_size).to('cuda')
    g.ndata['params'] = torch.tensor(node_params).to('cuda')
    g.ndata['bias'] = torch.stack(all_bias_feats)
    g.ndata['bias_size'] = torch.tensor(pre_bias_size).to('cuda')
    
    return g

# from model_lib import mnist_cnn_model as father_model
from utils_basic import load_spec_model
from utils_gnn import padding
x = '/home/dorian/repos/Meta-Nerual-Trojan-Detection/shadow_model_ckpt/mnist/models5/shadow_jumbo_9.model'
# load model 
# Model = load_spec_model(father_model, '5')
from model_lib.mnist_cnn_model import Model6 as Model
model = Model().cuda()
params = torch.load(x)
model.load_state_dict(params)

# load model detail 
model_detail = {}
model_detail_path = "./intermediate_data/model_detail.json"
import json
with open(model_detail_path, 'r') as f:
    model_detail = json.load(f)
# print(model_detail)
g = cnn2graph_activation(model, model_detail['mnist']['5'])
dgl.save_graphs('./intermediate_data/grapj_test.bin', g)

In [3]:
from dgl.dataloading import GraphDataLoader
from dgl.heterograph import DGLHeteroGraph
from model_lib.hetero_struct_backdoor_dataset import HeteroStrucBackdoorDataset
import torch
def test_hetero_activation():
    dataset = HeteroStrucBackdoorDataset(nums=10,
        raw_dir='/home/dorian/repos/Meta-Nerual-Trojan-Detection/shadow_model_ckpt/mnist/models', activation=True)
    dataloader = GraphDataLoader(dataset, batch_size=1, pin_memory=torch.cuda.is_available())
    for batch, (batched_graph, labels) in enumerate(dataloader):
        assert batch == 0
        assert isinstance(batched_graph, DGLHeteroGraph)
        # assert batched_graph.ndata['idx'].any()
        # assert batched_graph.ndata['node_size']
        # assert batched_graph.ndata['params']
        # assert batched_graph.ndata['bias']
        # assert batched_graph.ndata['bias_size']
        print(batched_graph)
        break

test_hetero_activation() 


Graph(num_nodes=162, num_edges=5153,
      ndata_schemes={'x': Scheme(shape=(512, 512), dtype=torch.float32), 'idx': Scheme(shape=(), dtype=torch.int64), 'node_size': Scheme(shape=(2,), dtype=torch.int64), 'params': Scheme(shape=(3, 2), dtype=torch.int64), 'bias': Scheme(shape=(1, 512), dtype=torch.float32), 'bias_size': Scheme(shape=(2,), dtype=torch.int64)}
      edata_schemes={})


### 2. 设计传播函数模拟神经网络传参
- 图片分类网络结果  
    - [ ] 读取数据
    - [ ] 调用分类网络获得分类结果
    - [ ] 获取网络中间结果  
- 设计传播函数  
    - [ ] conv 层
    - [ ] 全连接层
    - [ ] 输出层获得分类结果

In [6]:
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose([
            transforms.ToTensor(),
        ])
BATCH_SIZE = 1
trainset = torchvision.datasets.MNIST(root='./raw_data/', train=True, download=True, transform=transform)
dataloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE)

print(model.name)
for i in enumerate(dataloader):
    print(i)
    break

from model_lib.hetero_struct_backdoor_dataset import HeteroStrucBackdoorDataset
dataset = HeteroStrucBackdoorDataset(nums=10, activation=True, raw_dir='./shadow_model_ckpt/mnist/models/')
print(len(dataset))

Model6


TypeError: 'module' object is not callable